# H_22082024
# Analisis del volumen profile para encontrar zonas de baja cotización (FVG like) y como se comporta el precio en esas zonas.

En este Python Notebook analizaremos, con datos de mt5, como se comporta el mercado en las zonas previamente definidas.

## Conceptos clave del perfil de mercado:

1. **Oportunidad de precio en el tiempo (TPO):**
    
    representa los niveles de precio negociados durante intervalos de tiempo específicos. En un gráfico de perfil de mercado, las TPO indican la frecuencia con la que se negoció cada nivel de precio durante una sesión.
    
2. **Área de valor:**
    
    rango de precios en el que se produjo el 70 % de la actividad comercial durante una sesión. Esta área representa el valor justo percibido del mercado.
    
3. **Punto de control (POC):**
    
    el nivel de precio con el mayor volumen durante la sesión. El POC suele ser un nivel clave de soporte o resistencia.
    
4. **Balance inicial (IB):**
    
    rango de precios negociados durante la primera hora de la sesión de negociación. Proporciona una visión anticipada de la estructura del mercado del día.

In [1]:
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
import pytz
from datetime import datetime
import matplotlib.pyplot as plt
import json
import plotly.express as px
import plotly.graph_objects as go

In [12]:
# Variables

TICKER = "GC_V"
STD_MULTIPLIER = 1

## 1. Definición de zonas de baja cotización para posterior analisis.
El primer paso a dar es definir como encontraremos las **zonas de baja cotización** en una sesion de mercado de forma objetiva.

A continuación, definiremos que es una zona de baja cotización para nuestro estudio:

1. Analisis basado en el *volumen profile*, el cual analiza el numero de cotizaciónes por precio que tiene lugar en una sesión de trading predefinida (en nuestro estudio, una sesión será un dia 1D).
2. La media y la StdDev del volumen profile de la sesión nos ayudarán a analizar correctamente las zonas.

Por tanto, una **zona de baja cotización** consiste en:
- Los niveles de precio que se encuentran N*StdDev(mean(volume_profile)) por debajo.

In [3]:
def data_from_mt5():
    # connect to MetaTrader 5
    if not mt5.initialize():
        print("initialize() failed")
        mt5.shutdown()

    # set time zone to UTC
    timezone = pytz.timezone("Etc/UTC")
    # create 'datetime' objects in UTC time zone to avoid the implementation of a local time zone offset
    utc_from = datetime(2024, 5, 1, tzinfo=timezone)
    utc_to = datetime(2024, 6, 30, tzinfo=timezone)

    # request AUDUSD ticks within 11.01.2020 - 11.01.2020
    ohlcv = mt5.copy_rates_range(TICKER, mt5.TIMEFRAME_M1, utc_from, utc_to)
    daily_ohlcv = mt5.copy_rates_range(TICKER, mt5.TIMEFRAME_D1, utc_from, utc_to)

    mt5.shutdown()

    df = pd.DataFrame(ohlcv)
    df['time']=pd.to_datetime(df['time'], unit='s')

    # adaptamos el dataframe D1 para luego hacer los analisis de las sesiones
    daily_df = pd.DataFrame(daily_ohlcv)
    daily_df['time']=pd.to_datetime(daily_df['time'], unit='s')

    df = df.set_index('time')
    del df['real_volume']
    del df['spread']

    daily_df = daily_df.set_index('time')
    del daily_df['real_volume']
    del daily_df['spread']

    return df, daily_df

def data_from_csv():
    df = pd.read_csv('C:/Users/iamfr/AlgoTrading/DATA/GC_V_M1_20091028_20240628.csv', sep='\t')
    df['<DATE>'] = pd.to_datetime(df['<DATE>'] + ' ' + df['<TIME>'])
    del df['<TIME>']
    del df['<VOL>']
    del df['<SPREAD>']
    df.columns = ['time', 'open','high', 'low', 'close', 'tick_volume']
    df = df.set_index('time')

    daily_df = pd.read_csv('C:/Users/iamfr/AlgoTrading/DATA/GC_V_D1_20091028_20240628.csv', sep='\t')
    #df['<DATE>'] = pd.to_datetime(df['<DATE>'] + ' ' + df['<TIME>'])
    #del df['<TIME>']
    del daily_df['<VOL>']
    del daily_df['<SPREAD>']
    daily_df.columns = ['time', 'open','high', 'low', 'close', 'tick_volume']
    daily_df = daily_df.set_index('time')
    #daily_df = df.resample('1B').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'tick_volume': 'sum'})

    return df, daily_df

df, daily_df = data_from_csv()

In [4]:
def create_market_profile(data):
    profile = data.groupby('close')['tick_volume'].sum().reset_index()
    total_volume = profile['tick_volume'].sum()
    profile['volume_cumsum'] = profile['tick_volume'].cumsum()

    value_area_cutoff = total_volume * 0.70
    value_area_df = profile[profile['volume_cumsum'] <= value_area_cutoff]
    POC = profile.loc[profile['tick_volume'].idxmax(), 'close']

    return profile, value_area_df, POC

def plot_market_profile(profile, value_area_df, POC):
    plt.figure(figsize=(10, 6))
    plt.barh(profile['close'], profile['tick_volume'], color='blue', edgecolor='black')
    plt.barh(value_area_df['close'], value_area_df['tick_volume'], color='green', edgecolor='black')
    plt.axhline(POC, color='red', linestyle='--', label=f'POC: {POC}')

    plt.xlabel('Volume')
    plt.ylabel('Price')
    plt.title(f'Market Profile')
    plt.legend()
    plt.show()

In [5]:
def get_no_fair_range_zone(df, threshold):
    # Crear una máscara booleana para identificar dónde 'tick_volume' es inferior al umbral
    df.sort_values(by=['close'])
    mask = df['tick_volume'] < threshold

    # Encontrar los índices donde empieza y termina cada zona
    rangos = []
    inicio = None
    rsize = 0
    rango_max = []

    for i in range(len(df)):
        if mask[i]:
            if inicio is None:  # Se inicia una nueva zona
                inicio = i
        else:
            if inicio is not None:  # Se cierra la zona actual
                rangos.append([inicio, i - 1])
                inicio = None

    # Si la última zona no se cierra explícitamente en el bucle
    if inicio is not None:
        rangos.append([inicio, len(df) - 1])
                 
    for rango in rangos:
        if rsize <= (rango[1] - rango[0]):  
              rsize = rango[1] - rango[0]
              rango_max = rango

    if len(rangos) < 1:
        return False
    return [df.loc[rango_max[0], 'close'], df.loc[rango_max[1], 'close']]

def get_max_vol_zone(df, threshold):
    # Crear una máscara booleana para identificar dónde 'tick_volume' es inferior al umbral
    df.sort_values(by=['close'])
    mask = df['tick_volume'] > threshold

    # Encontrar los índices donde empieza y termina cada zona
    rangos = []
    inicio = None
    rsize = 0
    rango_max = []

    for i in range(len(df)):
        if mask[i]:
            if inicio is None:  # Se inicia una nueva zona
                inicio = i
        else:
            if inicio is not None:  # Se cierra la zona actual
                rangos.append([inicio, i - 1])
                inicio = None

    # Si la última zona no se cierra explícitamente en el bucle
    if inicio is not None:
        rangos.append([inicio, len(df) - 1])
                 
    for rango in rangos:
        if rsize <= (rango[1] - rango[0]):  
              rsize = rango[1] - rango[0]
              rango_max = rango

    if len(rangos) < 1:
        return False
    return [df.loc[rango_max[0], 'close'], df.loc[rango_max[1], 'close']]

In [6]:
# iteramos cada dia, y cada zona
def get_no_fair_zone_by_day(df):
    out = []

    for index1, day in df.groupby(df.index.date):

        profile, value_area_df, POC = create_market_profile(day)
        mean = profile['tick_volume'].mean()
        stddev = profile['tick_volume'].std()
        threshold = mean + stddev * STD_MULTIPLIER

        no_fair_value_zone = get_no_fair_range_zone(profile, threshold)
        
        output = {
            "time": index1, #.strftime("%Y-%m-%d")
            "zone_high": no_fair_value_zone[1],
            "zone_low": no_fair_value_zone[0],
        }
        out.append(output)

    return out

# iteramos cada dia, y cada zona
def get_max_vol_zone_by_day(df):
    out = []

    for index1, day in df.groupby(df.index.date):

        profile, value_area_df, POC = create_market_profile(day)
        mean = profile['tick_volume'].mean()
        stddev = profile['tick_volume'].std()
        threshold = mean + stddev * STD_MULTIPLIER

        no_fair_value_zone = get_max_vol_zone(profile, threshold)
        
        output = {
            "time": index1,
            "zone_high": no_fair_value_zone[1],
            "zone_low": no_fair_value_zone[0],
        }
        out.append(output)

    return out

## Analisis de datos

A partir de aqui, ya disponemos de las herramientas de manipulación de los datos, como para poder empezar a analizarlos.

Disponemos de una lista "ZONES_BY_DAY", la qual nos permitirá referenciar las zonas de baja cotización por dia, y sus niveles de precio. Asi podemos hacer un analisis más comodo.

In [13]:
#zones = get_no_fair_zone_by_day(df)
zones = get_max_vol_zone_by_day(df)
df_zones = pd.DataFrame(zones)
#df_zones['time'].astype('datetime64[ns]')
df_zones = df_zones.set_index('time')
df_zones.index.astype('datetime64[ns]')

main_df = pd.merge(daily_df, df_zones, on=daily_df.index) # Arreglar esto, porque me da problema
#daily_df.join(df_zones, on=daily_df.index , how='inner')
main_df = main_df.set_index('key_0')
main_df

,open,high,low,close,tick_volume,zone_high,zone_low
key_0,,,,,,,
2009.10.28,1372.6,1376.8,1357.7,1358.3,890,1371.8,1371.7
2009.10.29,1358.9,1384.8,1357.2,1384.2,920,1380.4,1380.4
2009.10.30,1383.7,1385.9,1368.6,1370.4,790,1384.7,1384.5
2009.10.31,1371.5,1383.5,1371.5,1382.0,512,1374.8,1374.8
2009.11.02,1379.2,1405.2,1379.2,1403.3,1144,1402.3,1401.6
...,...,...,...,...,...,...,...
2024.06.24,2354.2,2370.6,2353.0,2370.4,134044,2360.5,2359.9
2024.06.25,2369.5,2372.7,2350.5,2354.6,160721,2362.0,2361.3
2024.06.26,2354.5,2358.6,2327.6,2332.4,197254,2351.7,2351.2


### 1. Cierre de rango en la siguiente sesión

Analizaremos si en la siguiente sesión, el precio cierra el rango del dia anterior.

In [14]:
def num1_next_session_close_zone(df):
    counter = 0

    for i in range(len(df.index) - 2):
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        zone_high = df.iloc[i]['zone_high']
        zone_low = df.iloc[i]['zone_low']

        if zone_high <= high and zone_high >= low and zone_low >= low and zone_low <= high:
            counter = counter + 1

    return (counter / (len(df.index) - 1)) * 100

print("Porcentaje de cierre de la zona: ", num1_next_session_close_zone(main_df))

Porcentaje de cierre de la zona:  68.80108991825612


### 2. Cierre de medio rango en la siguiente sesion

Analizaremos si en la siguiente sesión, el precio cierra la midat del rango.

In [15]:
def num2_next_session_close_half_zone(df):
    counter = 0

    for i in range(len(df.index) - 2):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        zone_high = df.iloc[i]['zone_high']
        zone_low = df.iloc[i]['zone_low']
        zone_mid = ((zone_high - zone_low) / 2) + zone_low

        if apertura >= zone_high:
            # Mitad superior
            if zone_high <= high and zone_high >= low and zone_mid >= low and zone_mid <= high:
                counter = counter + 1

        if apertura <= zone_low:
            # Mitad inferior
            if zone_mid <= high and zone_mid >= low and zone_low >= low and zone_low <= high:
                counter = counter + 1

    return (counter / (len(df.index) - 1)) * 100

print("Porcentaje de cierre de media zona: ", num2_next_session_close_half_zone(main_df))

Porcentaje de cierre de media zona:  66.28065395095368


### 3. Analizar la máxima varianza negativa

Analisis de la cantidad de veces consecutivas que la estadistica no ha funcionado.

In [16]:
def cum_sum_max_negative_variance(df):
    return False

### 4. Analizar la máxima varianza negativa

Analisis de la cantidad de veces consecutivas que la estadistica no ha funcionado.